In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_train.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


We have our label which is the number, and our 783 pixels for black and white. Values go from 0 -> 255 so we can divide by 255 to do a quick version of the min max scaler 

In [5]:
# Doublecheck no null values in any of the columns

sum(df_train.isnull().sum())

0

In [6]:
# Split dataset into X, y

X = df_train.drop(columns='label')
y = df_train['label']

print(X.head(3))
print(y.head(3))

   pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  

[3 rows x 784 columns]
0    1
1    0
2    1
Name: label, dtype: int64


In [7]:
# Quick min max scaler
X = X/255

In [8]:
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Initial model creation



In [9]:
# Import packages

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

In [10]:
# Initialise Model
# We will have two layers with the relu activation function, and our final output layer with linear
# Since we will have the softmax function in the loss calculation

tf.random.set_seed(42)
model = Sequential(
    [
        tf.keras.Input(shape=(784,)),
        Dense(units=25, activation='relu'),
        Dense(units=15, activation='relu'),
        Dense(units=10, activation='linear'),
    ]
)

In [11]:
# See model summary

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 25)                  │          19,625 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │             390 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             160 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,175 (78.81 KB)

 Trainable params: 20,175 (78.81 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Let us compile

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [13]:
# Let us fit

history = model.fit(
    X,y,
    epochs=50,
    validation_split = 0.1
)

Epoch 1/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8889 - val_loss: 0.2877
Epoch 2/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.2744 - val_loss: 0.2439
Epoch 3/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.2234 - val_loss: 0.2128
Epoch 4/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1877 - val_loss: 0.1944
Epoch 5/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1628 - val_loss: 0.1820
Epoch 6/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1447 - val_loss: 0.1737
Epoch 7/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1298 - val_loss: 0.1699
Epoch 8/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1179 - val_loss: 0.1684
Epoch 9/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1088 - val_loss: 0.1678
Epoch 10/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1009 - val_loss: 0.1691
Epoch 11/50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0939 - val_loss: 0.1693
Epoch 12/50
1182/1182 ━━━━━━━━

In [14]:
# Let's visualise the accuracy over time

import keras
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss vs epochs')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'plt' is not defined

The cross validation error is diverging from the training error is, which implies that this network is overfitting some. Let's play with the regularisation to see what we can do here

# Introducing regularization to the model to address overfitting

Let us first try with an example regularization parameter, and then see if we can optimise it

In [ ]:
from tensorflow.keras.regularizers import L2

tf.random.set_seed(42)
model_reg_example = Sequential(
    [
        tf.keras.Input(shape=(784,)),
        Dense(units=25, activation='relu', kernel_regularizer=L2(0.01)),
        Dense(units=15, activation='relu', kernel_regularizer=L2(0.01)),
        Dense(units=10, activation='linear', kernel_regularizer=L2(0.01)),
    ]
)

In [ ]:
# See model summary

model_reg_example.summary()

In [ ]:
# Let us compile

model_reg_example.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [ ]:
# Let us fit

history_reg_example = model_reg_example.fit(
    X,y,
    epochs=50,
    validation_split = 0.1
)

In [ ]:
# Let's visualise the loss function over epochs

plt.plot(history_reg_example.history['loss'])
plt.plot(history_reg_example.history['val_loss'])
plt.title('regularized model loss vs epochs')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

There is a higher error in the training set, though it's closer to the CV error which implies the overfitting issue has been resolved

Let's see where we get with this initial play around and then think about ways to improve

# Predictions

In [ ]:
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
df_test.head()

In [ ]:
df_sample = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
df_sample.head()

In [ ]:
df_test_norm = df_test/255

In [ ]:
predictions = model_reg_example.predict(df_test_norm)

In [ ]:
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
predicted_labels

In [ ]:
submission = pd.DataFrame(
    {"ImageId": np.arange(1, len(predicted_labels) + 1), "Label": predicted_labels}
)

submission.to_csv("submission.csv", index=False)
print("Submission file created.")

# **Score: 0.93164**